In [103]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime

from dotenv import load_dotenv

import xlsxwriter

# import requests
# import json

# current date
date = datetime.now().strftime("%Y%m%d-%H%M")
print(date) 

20230123-2006


In [104]:
# Pull data from historic data in csv files
# link to data https://www.football-data.co.uk/mmz4281/2223/E0.csv 
# where 2223 is the year and E0 is the league

# read in data
df02223 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2223/E0.csv')
df12223 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2223/E1.csv')
df02122 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2122/E0.csv')
df12122 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2122/E1.csv')
df2021 = pd.read_csv('https://www.football-data.co.uk/mmz4281/2021/E0.csv')
df1920 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1920/E0.csv')
df1819 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1819/E0.csv')
df1718 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1718/E0.csv') 
df1617 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1617/E0.csv')
df1516 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1516/E0.csv')
df1415 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1415/E0.csv')
df1314 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1314/E0.csv')
df1213 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1213/E0.csv')
df1112 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1112/E0.csv')
df1011 = pd.read_csv('https://www.football-data.co.uk/mmz4281/1011/E0.csv')


# create a list of dataframes
df_list = [df02223, df12223, df02122, df12122, df2021, df1920, df1819, df1718, df1617, df1516, df1415, df1314, df1213, df1112, df1011]



In [153]:
# create list of seasons from 1993 to current year
# where year is 2 digits
# and the season is the year + the next year
seasons = []
for i in range(1993, 2023):
    seasons.append(str(i)[2:4] + str(i+1)[2:4])

# append the league to the season 
# in to format season/league.csv 
# where the league is E0, E1, E2, E3
# and create a list of seasons and leagues
seasons_leagues = []
for season in seasons:
    for league in ['E0', 'E1', 'E2', 'E3']:
        seasons_leagues.append(season + '/' + league + '.csv')

# add the path to the data "https://www.football-data.co.uk/mmz4281/
# to the list of seasons and leagues
seasons_leagues = ['https://www.football-data.co.uk/mmz4281/' + season_league for season_league in seasons_leagues]


print(seasons_leagues)




['https://www.football-data.co.uk/mmz4281/9394/E0.csv', 'https://www.football-data.co.uk/mmz4281/9394/E1.csv', 'https://www.football-data.co.uk/mmz4281/9394/E2.csv', 'https://www.football-data.co.uk/mmz4281/9394/E3.csv', 'https://www.football-data.co.uk/mmz4281/9495/E0.csv', 'https://www.football-data.co.uk/mmz4281/9495/E1.csv', 'https://www.football-data.co.uk/mmz4281/9495/E2.csv', 'https://www.football-data.co.uk/mmz4281/9495/E3.csv', 'https://www.football-data.co.uk/mmz4281/9596/E0.csv', 'https://www.football-data.co.uk/mmz4281/9596/E1.csv', 'https://www.football-data.co.uk/mmz4281/9596/E2.csv', 'https://www.football-data.co.uk/mmz4281/9596/E3.csv', 'https://www.football-data.co.uk/mmz4281/9697/E0.csv', 'https://www.football-data.co.uk/mmz4281/9697/E1.csv', 'https://www.football-data.co.uk/mmz4281/9697/E2.csv', 'https://www.football-data.co.uk/mmz4281/9697/E3.csv', 'https://www.football-data.co.uk/mmz4281/9798/E0.csv', 'https://www.football-data.co.uk/mmz4281/9798/E1.csv', 'https://

In [158]:
# load each of the csv files into a dataframe per file
# and append to a list of dataframes
df_list = []
for season_league in seasons_leagues:
    df_list.append(pd.read_csv(season_league))

# create a list of dataframes
df_list = [df02223, df12223, df02122, df12122, df2021, df1920, df1819, df1718, df1617, df1516, df1415, df1314, df1213, df1112, df1011]


ParserError: Error tokenizing data. C error: Expected 28 fields in line 17, saw 45


In [135]:
# print unique values in the div column
print(df['Div'].unique())

['E0' 'E1' nan]


In [105]:
# concat dataframes into one dataframe
df = pd.concat(df_list, ignore_index=True)
# replace 4 digit year with 2 digit year
df['Date'] = df['Date'].str.replace('2023', '23')
df['Date'] = df['Date'].str.replace('2022', '22')
df['Date'] = df['Date'].str.replace('2021', '21')
df['Date'] = df['Date'].str.replace('2020', '20')
df['Date'] = df['Date'].str.replace('2019', '19')  
df['Date'] = df['Date'].str.replace('2018', '18')
df['Date'] = df['Date'].str.replace('2017', '17')
df['Date'] = df['Date'].str.replace('2016', '16')
df['Date'] = df['Date'].str.replace('2015', '15')
df['Date'] = df['Date'].str.replace('2014', '14')
df['Date'] = df['Date'].str.replace('2013', '13')
df['Date'] = df['Date'].str.replace('2012', '12')
df['Date'] = df['Date'].str.replace('2011', '11')


df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA,SJH,SJD,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA
0,E0,05/08/22,20:00,Crystal Palace,Arsenal,0.0,2.0,A,0.0,1.0,A,A Taylor,10.0,10.0,2.0,2.0,16.0,11.0,3.0,5.0,1.0,2.0,0.0,0.0,4.20,3.6,1.85,4.33,3.50,1.87,4.30,3.55,1.85,4.50,3.65,1.89,4.40,3.5,1.83,4.60,3.5,1.87,4.60,3.78,1.95,4.39,3.59,1.88,2.10,1.72,2.14,1.78,2.19,1.91,2.09,1.76,0.50,2.04,1.89,2.03,1.89,2.06,1.91,2.01,1.87,4.50,3.60,1.80,4.50,3.5,1.83,4.40,3.55,1.85,4.58,3.63,1.88,4.80,3.4,1.78,4.75,3.5,1.85,5.01,3.70,1.91,4.56,3.57,1.85,2.10,1.72,2.14,1.78,2.19,1.91,2.08,1.76,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,06/08/22,12:30,Fulham,Liverpool,2.0,2.0,D,1.0,0.0,H,A Madley,9.0,11.0,3.0,4.0,7.0,9.0,4.0,4.0,2.0,0.0,0.0,0.0,11.00,6.0,1.25,10.00,5.75,1.28,12.00,5.75,1.27,11.20,6.22,1.28,12.00,5.5,1.27,13.00,6.0,1.25,13.00,6.40,1.31,10.99,6.05,1.28,1.50,2.62,1.50,2.70,1.54,2.76,1.48,2.63,1.75,1.90,2.03,1.91,2.00,1.92,2.04,1.89,1.99,11.00,5.75,1.28,9.25,6.0,1.29,11.00,5.50,1.30,10.50,6.50,1.29,11.00,5.5,1.27,11.50,6.0,1.29,11.95,6.93,1.30,10.33,6.20,1.28,1.50,2.62,1.49,2.77,1.51,3.00,1.47,2.73,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,06/08/22,15:00,Bournemouth,Aston Villa,2.0,0.0,H,1.0,0.0,H,P Bankes,7.0,15.0,3.0,2.0,18.0,16.0,5.0,5.0,3.0,3.0,0.0,0.0,3.75,3.5,2.00,3.75,3.40,2.00,3.65,3.45,2.05,3.93,3.58,2.04,3.75,3.3,2.05,3.75,3.3,2.00,4.00,3.66,2.10,3.80,3.50,2.04,2.00,1.80,2.10,1.81,2.10,1.87,2.03,1.80,0.50,1.87,2.06,1.88,2.04,1.88,2.07,1.85,2.04,4.00,3.50,1.95,3.90,3.4,1.95,3.85,3.45,2.00,4.09,3.59,2.00,4.00,3.4,1.95,4.10,3.4,2.00,4.25,3.63,2.06,3.99,3.49,2.00,2.10,1.72,2.13,1.79,2.24,1.81,2.10,1.76,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,06/08/22,15:00,Leeds,Wolves,2.0,1.0,H,1.0,1.0,D,R Jones,12.0,15.0,4.0,6.0,13.0,9.0,6.0,4.0,2.0,0.0,0.0,0.0,2.25,3.4,3.20,2.30,3.30,2.95,2.30,3.30,3.15,2.39,3.33,3.30,2.25,3.3,3.20,2.30,3.2,3.10,2.42,3.54,3.30,2.34,3.34,3.18,2.05,1.85,2.09,1.83,2.11,1.87,2.03,1.81,-0.25,2.05,1.88,2.04,1.89,2.06,1.90,2.01,1.87,2.37,3.30,3.00,2.40,3.3,2.75,2.45,3.30,2.95,2.45,3.44,3.09,2.40,3.3,2.90,2.40,3.4,3.00,2.50,3.55,3.18,2.43,3.36,3.02,1.95,1.95,1.96,1.94,2.09,1.96,1.96,1.87,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,06/08/22,15:00,Newcastle,Nott'm Forest,2.0,0.0,H,0.0,0.0,D,S Hooper,23.0,5.0,10.0,0.0,9.0,14.0,11.0,1.0,0.0,3.0,0.0,0.0,1.66,3.8,5.25,1.65,3.80,5.50,1.65,3.80,5.50,1.71,3.74,5.83,1.67,3.7,5.25,1.62,3.7,5.50,1.72,3.96,6.00,1.67,3.80,5.57,2.05,1.85,2.10,1.81,2.10,1.86,2.03,1.81,-0.75,1.87,2.06,1.92,2.01,1.92,2.08,1.86,2.02,1.53,4.00,6.00,1.58,3.9,6.00,1.63,3.80,6.00,1.57,4.22,6.60,1.53,3.9,6.50,1.57,3.9,7.00,1.67,4.30,7.00,1.59,4.07,6.15,1.94,1.96,1.97,1.93,2.06,1.97,1.94,1.89,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [106]:
# change values in Div column 
df['Div'] = df['Div'].str.replace('E0', 'Premier League')
df['Div'] = df['Div'].str.replace('E1', 'Championship')
df['Div'] = df['Div'].str.replace('E2', 'League One')
df['Div'] = df['Div'].str.replace('E3', 'League Two')

# convert date column to datetime with multiple formats
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y')

# sort by date and time
df = df.sort_values(by=['Date', 'Time'], ascending=True)
# reindex
df = df.reset_index(drop=True)
# add id column
df['id'] = df.index


print(df.shape)
df.head()

(5644, 140)


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,LBH,LBD,LBA,SJH,SJD,SJA,GBH,GBD,GBA,BSH,BSD,BSA,SBH,SBD,SBA,id
0,Premier League,2010-08-14,NaN,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,M Dean,23.0,12.0,11.0,2.0,15.0,15.0,16.0,7.0,1.0,2.0,0.0,0.0,2.00,3.30,4.0,1.85,3.40,4.10,1.80,3.4,4.0,NaN,NaN,NaN,1.91,3.25,4.33,2.00,3.25,4.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,2.09,1.96,3.40,3.30,4.33,4.03,35.0,2.11,2.01,1.82,1.75,20.0,0.0,1.45,1.40,3.00,2.78,1.83,3.4,4.33,2.00,3.4,3.75,2.00,3.25,3.75,1.80,3.40,4.33,1.90,3.25,3.75,0
1,Premier League,2010-08-14,NaN,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,P Dowd,7.0,17.0,2.0,12.0,19.0,14.0,1.0,3.0,2.0,1.0,0.0,0.0,2.88,3.25,2.5,2.75,3.25,2.45,2.55,3.2,2.5,NaN,NaN,NaN,2.88,3.20,2.50,3.10,3.30,2.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,3.15,2.92,3.40,3.25,2.55,2.44,36.0,2.20,2.08,1.79,1.70,29.0,0.0,2.20,2.05,1.85,1.75,2.80,3.3,2.50,2.75,3.3,2.50,2.90,3.20,2.40,2.70,3.25,2.50,2.70,3.20,2.40,1
2,Premier League,2010-08-14,NaN,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,S Attwell,13.0,12.0,9.0,7.0,12.0,13.0,4.0,8.0,1.0,3.0,0.0,0.0,2.20,3.30,3.4,2.15,3.25,3.25,2.20,3.2,3.0,NaN,NaN,NaN,2.15,3.20,3.50,2.20,3.30,3.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,2.30,2.20,3.42,3.26,3.58,3.32,35.0,2.22,2.09,1.75,1.69,24.0,0.0,1.60,1.55,2.56,2.36,2.25,3.3,3.20,2.30,3.4,3.00,2.20,3.25,3.20,2.20,3.20,3.25,2.10,3.20,3.20,2
3,Premier League,2010-08-14,NaN,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,M Clattenburg,18.0,10.0,13.0,4.0,10.0,10.0,3.0,1.0,1.0,0.0,0.0,0.0,1.17,7.00,17.0,1.15,6.75,16.00,1.15,6.2,14.0,NaN,NaN,NaN,1.17,6.50,19.00,1.17,7.50,19.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,1.19,1.16,7.71,6.90,21.00,17.47,29.0,1.53,1.49,2.61,2.49,19.0,-2.0,2.06,2.01,1.91,1.85,1.14,7.5,19.00,1.18,6.5,15.00,1.16,6.75,16.00,1.17,6.50,15.00,1.12,6.50,17.00,3
4,Premier League,2010-08-14,NaN,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,A Taylor,6.0,13.0,2.0,7.0,13.0,10.0,3.0,6.0,3.0,3.0,1.0,0.0,2.10,3.30,3.6,2.15,3.20,3.30,2.10,3.2,3.2,NaN,NaN,NaN,2.15,3.20,3.50,2.20,3.25,3.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,2.31,2.18,3.40,3.25,3.60,3.39,36.0,2.28,2.16,1.71,1.65,24.0,0.0,1.64,1.55,2.50,2.38,2.10,3.3,3.50,2.20,3.4,3.20,2.15,3.25,3.30,2.10,3.25,3.40,2.20,3.20,3.00,4


In [107]:
# export to excel
df.to_excel(f'../../output/{date}-historic.xlsx', index=False)

In [115]:

points_map = {
    'W': 3,
    'D': 1,
    'L': 0
}

# Results Function
def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

In [119]:
df_matches['Opponent'] = np.where(
    df_matches['home_away'] == 'H', 
    df_matches['AwayTeam'], 
    df_matches['HomeTeam']
    )

# create a column for goals scored
df_matches['goals_scored'] = np.where(
    df_matches['home_away'] == 'H',
    df_matches['FTHG'],
    df_matches['FTAG']
    )
# create a column for goals conceded
df_matches['goals_conceded'] = np.where(
    df_matches['home_away'] == 'H',
    df_matches['FTAG'],
    df_matches['FTHG']
    )

# create a column for result
df_matches['result'] = np.vectorize(get_result)(
    df_matches['goals_scored'], df_matches['goals_conceded'])

# create a column for points
df_matches['points'] = df_matches['result'].map(points_map)

# sort by date descending and id ascending
df_matches = df_matches.sort_values(by=['Date', 'id'], ascending=[False, True])

# add a season column, season starts in August
df_matches['season'] = np.where(
    df_matches['Date'].dt.month >= 8,
    df_matches['Date'].dt.year,
    df_matches['Date'].dt.year - 1
    )

# reset index
df_matches = df_matches.reset_index(drop=True)

df_matches.head(30)


,id,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,home_away,team,Opponent,goals_scored,goals_conceded,result,points,season
0,5639,Championship,2023-01-22,Sunderland,Middlesbrough,2,0,H,H,Sunderland,Middlesbrough,2,0,W,3,2022
1,5639,Championship,2023-01-22,Sunderland,Middlesbrough,2,0,H,A,Middlesbrough,Sunderland,0,2,L,0,2022
2,5640,Premier League,2023-01-22,Leeds,Brentford,0,0,D,H,Leeds,Brentford,0,0,D,1,2022
3,5640,Premier League,2023-01-22,Leeds,Brentford,0,0,D,A,Brentford,Leeds,0,0,D,1,2022
4,5641,Premier League,2023-01-22,Man City,Wolves,3,0,H,H,Man City,Wolves,3,0,W,3,2022
5,5641,Premier League,2023-01-22,Man City,Wolves,3,0,H,A,Wolves,Man City,0,3,L,0,2022
6,5642,Premier League,2023-01-22,Arsenal,Man United,3,2,H,H,Arsenal,Man United,3,2,W,3,2022
7,5642,Premier League,2023-01-22,Arsenal,Man United,3,2,H,A,Man United,Arsenal,2,3,L,0,2022
8,5625,Premier League,2023-01-21,Liverpool,Chelsea,0,0,D,H,Liverpool,Chelsea,0,0,D,1,2022
9,5625,Premier League,2023-01-21,Liverpool,Chelsea,0,0,D,A,Chelsea,Liverpool,0,0,D,1,2022


In [115]:

points_map = {
    'W': 3,
    'D': 1,
    'L': 0
}

# Results Function
def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

In [119]:
df_matches['Opponent'] = np.where(
    df_matches['home_away'] == 'H', 
    df_matches['AwayTeam'], 
    df_matches['HomeTeam']
    )

# create a column for goals scored
df_matches['goals_scored'] = np.where(
    df_matches['home_away'] == 'H',
    df_matches['FTHG'],
    df_matches['FTAG']
    )
# create a column for goals conceded
df_matches['goals_conceded'] = np.where(
    df_matches['home_away'] == 'H',
    df_matches['FTAG'],
    df_matches['FTHG']
    )

# create a column for result
df_matches['result'] = np.vectorize(get_result)(
    df_matches['goals_scored'], df_matches['goals_conceded'])

# create a column for points
df_matches['points'] = df_matches['result'].map(points_map)

# sort by date descending and id ascending
df_matches = df_matches.sort_values(by=['Date', 'id'], ascending=[False, True])

# add a season column, season starts in August
df_matches['season'] = np.where(
    df_matches['Date'].dt.month >= 8,
    df_matches['Date'].dt.year,
    df_matches['Date'].dt.year - 1
    )

# reset index
df_matches = df_matches.reset_index(drop=True)

df_matches.head(30)


,id,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,home_away,team,Opponent,goals_scored,goals_conceded,result,points,season
0,5639,Championship,2023-01-22,Sunderland,Middlesbrough,2,0,H,H,Sunderland,Middlesbrough,2,0,W,3,2022
1,5639,Championship,2023-01-22,Sunderland,Middlesbrough,2,0,H,A,Middlesbrough,Sunderland,0,2,L,0,2022
2,5640,Premier League,2023-01-22,Leeds,Brentford,0,0,D,H,Leeds,Brentford,0,0,D,1,2022
3,5640,Premier League,2023-01-22,Leeds,Brentford,0,0,D,A,Brentford,Leeds,0,0,D,1,2022
4,5641,Premier League,2023-01-22,Man City,Wolves,3,0,H,H,Man City,Wolves,3,0,W,3,2022
5,5641,Premier League,2023-01-22,Man City,Wolves,3,0,H,A,Wolves,Man City,0,3,L,0,2022
6,5642,Premier League,2023-01-22,Arsenal,Man United,3,2,H,H,Arsenal,Man United,3,2,W,3,2022
7,5642,Premier League,2023-01-22,Arsenal,Man United,3,2,H,A,Man United,Arsenal,2,3,L,0,2022
8,5625,Premier League,2023-01-21,Liverpool,Chelsea,0,0,D,H,Liverpool,Chelsea,0,0,D,1,2022
9,5625,Premier League,2023-01-21,Liverpool,Chelsea,0,0,D,A,Chelsea,Liverpool,0,0,D,1,2022
